In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sc
import matplotlib.pyplot as plt
import yaml


In [3]:


class ParticleBasedMethod:
    def __init__(self, num_particles):
        self.num_particles = num_particles
        self.list_of_param = ['a', 'b']
        self.param_to_infer = len(self.list_of_param)
        self.variances_to_infer = 1

        self.prior_mean_a = 15
        self.prior_mean_std = 1
        self.prior_mean_b = 5
        self.prior_stddev_a_b = 1
        self.prior_stddev_std = 0.1


        self.particles = None
        self.weights = None
        self.initialize_particles()

        self.total_particles = self.particles


        # Current model and model parameters
        self.x_array = np.arange(1, 1001, 1)
        self.x_vect_ref = self.get_x_vect_ref()
        self.y_likelihood = 0



    def initialize_particles(self):
        self.particles = np.zeros((self.num_particles, self.param_to_infer+self.variances_to_infer))
        self.weights = np.ones(self.num_particles) / self.num_particles
        for i in range(self.num_particles):
            new = self.prior_sampler()
            for j in range(self.particles[i].size):
                self.particles[i][j] = new[j]

    def update_particles(self):


        temp_particles = np.zeros((self.num_particles, self.param_to_infer+self.variances_to_infer))
        temp_weights = np.ones(self.num_particles) / self.num_particles
        
        temp_particles = np.zeros((self.num_particles, self.param_to_infer+self.variances_to_infer))
        temp_weights = np.ones(self.num_particles) / self.num_particles
        for i in range(self.num_particles):
            temp_particles[i] = self.proposal_sampler(self.particles[i])
            #print( 'Likelihood:', self.likelihood_evaluator(self.particles[i]))
            temp_weights[i] *= self.likelihood_evaluator(self.particles[i])    
        
        self.particles = temp_particles
        self.weights = temp_weights

        self.weights /= np.sum(self.weights)
        #print('Weights: ', self.weights)

    '''def update_particles(self):
        temp_particles = np.zeros((self.num_particles, self.param_to_infer+self.variances_to_infer))
        temp_weights = np.ones(self.num_particles) / self.num_particles

        for i in range(self.num_particles):
            self.particles[i] = self.proposal_sampler(self.particles[i])
            #print( 'Likelihood:', self.likelihood_evaluator(self.particles[i]))
            self.weights[i] *= self.likelihood_evaluator(self.particles[i])    
        
        if(np.sum(self.weights)== 0):

        self.weights /= np.sum(self.weights)
        #print('Weights: ', self.weights)'''

    def resample_particles(self):
        indices = np.random.choice(range(self.num_particles), size=self.num_particles, replace=True, p=self.weights) 
        self.particles = self.particles[indices]
        self.weights = np.ones(self.num_particles) / self.num_particles


    # Define the prior distribution sampler
    def prior_sampler(self):
        # Implement your own prior distribution sampling logic
        # Return a sample from the prior distribution                
        sample_a = np.random.normal(self.prior_mean_a, self.prior_stddev_a_b, size= 1)
        sample_b = np.random.normal(self.prior_mean_b, self.prior_stddev_a_b, size= 1)
        sample_std = np.random.normal(self.prior_mean_std, self.prior_stddev_std, size= 1)
        sample = [sample_a, sample_b, sample_std]
        return sample
    
    # Define the proposal distribution sampler
    def proposal_sampler(self, particle):
        proposal_mean = particle  # Use the current particle as the mean for the proposal distribution
        proposal_stddev = 0.5 # Choose an appropriate standard deviation for the proposal distribution
        sample = np.random.normal(proposal_mean, proposal_stddev, size=len(particle))
        return sample

    # Define the likelihood evaluator
    def likelihood_evaluator(self, particle):
        
        # Given the observed data and a particle, compute the likelihood
        predicted_kpi = self.get_kpi(particle)  # Replace `model_simulation` with your own function that generates predicted data
        # Compute the likelihood using a probability distribution or a statistical measure
        if self.variances_to_infer == 1:
            log_likelihood_vect = sc.norm.logpdf(self.y_likelihood, loc= predicted_kpi, scale=particle[self.param_to_infer]) # Loc will be simulation(theta(:param_to_infer), scale = theta[param_to_infer])
            #print('log_likelihood_vect', log_likelihood_vect)
        else:
            mean_vector = np.full(self.param_to_infer, predicted_kpi)
            log_likelihood_vect = sc.multivariate_normal.logpdf(self.y_likelihood, mean= mean_vector, cov=np.diag(particle[self.param_to_infer:])) # Loc will be simulation(theta(:param_to_infer), scale = theta[param_to_infer])
        log_likelihood_out = np.sum(log_likelihood_vect)
        print(log_likelihood_out)
        likelihood = np.exp(log_likelihood_out)
        
        return likelihood
    #--------------- Open loop simulation ---------------#


    
    def get_kpi(self, particle):
        a = particle[0]
        b = particle[1]
        x_vect = self.get_open_loop_data(a,b)
        
        l2_norm_position = np.linalg.norm(self.x_vect_ref - x_vect, axis=1).sum()

        kpi = 0.00001*l2_norm_position
        return kpi


    def get_open_loop_data(self,a,b):
        res_vect = a*self.x_array +np.full(self.x_array.size, b)
        return res_vect.reshape(-1,1)
    
    def get_x_vect_ref(self):
        a = 10
        b = 5
        return self.get_open_loop_data(a,b)

    #
    def run_inference(self, num_iterations):
        for i in range(num_iterations):
            self.update_particles()
            self.resample_particles()

            print('Iteration: ',i)

        return self.particles
    
        

# Instantiate the particle-based method
num_particles = 200  # Number of particles to use
particle_method = ParticleBasedMethod(num_particles)

# Run the inference
num_iterations = 1000  # Number of iterations
particles = particle_method.run_inference(num_iterations)


-600.5632564211385
-118.32162486452702
-209.19889712867683
-284.32565938650816
-233.2119002560881
-409.36568490587007
-249.14961461529344
-293.0519079469822
-471.29154988914803
-319.8253391284396
-608.8913388322551
-328.98570545158134
-419.22419662274973
-222.64198974423644
-256.0031076898082
-269.39652325013685
-540.7283889153063
-183.7276017278947
-303.6822516445232
-298.9308867548526
-362.3779893336824
-200.19117983348102
-210.8867723799607
-192.25226224188484
-563.6045637973691
-398.5577257683061
-340.9973284827221
-454.27458696218383
-252.74362573197595
-166.77135415896
-420.243256455068
-365.3891968348659
-375.4099651309565
-345.6987001513942
-168.32954765454915
-221.54931143993645
-179.2254094086746
-257.5787460425694
-413.01209520040146
-391.7290784599006
-369.21375918580077
-237.62846968271123
-226.58461185910429
-204.2928430319326
-534.0657083449125
-131.47573744032286
-493.72922664631255
-313.08712940222597
-215.11938776641
-201.1751273046643
-202.5929629730911
-241.61003009

ValueError: probabilities contain NaN

In [67]:
particle_method.particles



array([[ 1.06026485e+01,  4.73122148e+00,  1.03206055e+00],
       [ 1.25729352e+01,  5.03307432e+00, -8.96027036e-02],
       [ 1.12104481e+01,  4.62320492e+00,  2.06019286e+00],
       [ 1.19580921e+01,  5.35907269e+00,  5.38368942e-01],
       [ 1.14591862e+01,  3.44025914e+00,  1.68623622e+00],
       [ 1.23361460e+01,  5.28650412e+00,  1.41874533e+00],
       [ 1.12084949e+01,  5.35778513e+00,  7.91273371e-01],
       [ 1.03687882e+01,  4.58135570e+00,  7.24086597e-01],
       [ 1.17295884e+01,  4.04482530e+00,  1.95188727e-01],
       [ 1.21665204e+01,  4.74011955e+00,  1.01825481e+00],
       [ 1.11472611e+01,  4.73525607e+00,  1.16136398e+00],
       [ 1.18144570e+01,  3.79367090e+00,  2.31143150e-01],
       [ 1.11014831e+01,  4.93997709e+00,  7.74755440e-01],
       [ 1.09786472e+01,  4.90544306e+00,  1.93731111e+00],
       [ 1.12419353e+01,  4.05813559e+00,  7.59616830e-01],
       [ 1.20224030e+01,  4.65877970e+00,  1.12274225e+00],
       [ 1.24030118e+01,  5.11361937e+00